In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Example").getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
df = spark.read.format("libsvm").load("sample_libsvm_data.txt")

In [5]:
df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
log_reg_model = LogisticRegression()

In [7]:
fitted_logreg = log_reg_model.fit(df)

In [8]:
log_summary = fitted_logreg.summary

In [9]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [10]:
log_summary.predictions.show()#note that label and prediction are perfect fitting since all data used for training

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000486...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716171...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298141...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113070...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823781...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446123...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [11]:
lr_train,lr_test = df.randomSplit([0.7,0.3])

In [12]:
final_model = LogisticRegression()

In [13]:
fit_final = final_model.fit(lr_train)

In [14]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [16]:
prediction_and_labels.predictions.show()# prediction and label are not perfectly fitting since this is test data

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[27.0529255139094...|[0.99999999999821...|       0.0|
|  0.0|(692,[98,99,100,1...|[31.6118453515492...|[0.99999999999998...|       0.0|
|  0.0|(692,[123,124,125...|[43.9025862153865...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[106.031284965654...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[41.8599154830667...|           [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[23.6032018639472...|[0.99999999994386...|       0.0|
|  0.0|(692,[151,152,153...|[40.4584644453273...|           [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|[12.1816715180096...|[0.99999487651989...|       0.0|
|  0.0|(692,[152,153,154...|[42.3850734663875...|           [1.0,0.0]|       0.0|
|  0.0|(692,[152

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [20]:
my_eva = BinaryClassificationEvaluator()

In [21]:
my_final_roc = my_eva.evaluate(prediction_and_labels.predictions)

In [22]:
my_final_roc## roc means area under the curve, if it is 1 means that perfect fit, graph showing the performance of a classification model at all classification thresholds

0.8666666666666667